# Hacking

Una gran empresa de tecnología necesita tu ayuda, ¡ha sido pirateada! Afortunadamente, sus ingenieros forenses obtuvieron datos valiosos sobre los ataques, incluida información como la hora de la sesión, las ubicaciones, la velocidad de escritura de palabras por minuto, etc. datos de cada sesión que los hackers utilizaron para conectarse a sus servidores. Estas son las características de los datos:


- 'Session_Connection_Time': cuánto duró la sesión en minutos
- 'Bytes transferidos': número de MB transferidos durante la sesión
- 'Kali_Trace_Used': Indica si el hacker estaba usando Kali Linux
- 'Servers_Corrupted': número de servidores dañados durante el ataque
- 'Pages_Corrupted': número de páginas a las que se ha accedido ilegalmente
- 'Ubicación': ubicación de dónde provino  el ataque (probablemente inútil porque los piratas informáticos usaron VPN)
- 'WPM_Typing_Speed': su velocidad de escritura estimada según los registros de sesión.

La firma de tecnología tiene 3 hackers potenciales que perpetraron el ataque. Están seguros de los dos primeros piratas informáticos, pero no están muy seguros de si el tercer pirata informático estuvo involucrado o no. ¡Han solicitado tu ayuda! ¿Puedes ayudar a averiguar si el tercer sospechoso tuvo algo que ver con los ataques o si fueron solo dos piratas informáticos? Probablemente no sea posible saberlo con certeza, pero tal vez lo que acaba de aprender sobre la agrupación en clústeres pueda ayudar.

Un último dato clave, el ingeniero forense sabe que los hackers intercambian ataques. Lo que significa que cada uno debería tener aproximadamente la misma cantidad de ataques. Por ejemplo, si hubo 100 ataques en total, entonces en una situación de 2 piratas informáticos, cada uno debería tener alrededor de 50 ataques, en una situación de tres piratas informáticos, cada uno tendría alrededor de 33 ataques. El ingeniero cree que este es el elemento clave para resolver esto, pero no sabe cómo distinguir estos datos sin etiquetar en grupos de piratas informáticos.

### Crear sesión

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('hacker').getOrCreate()

### Cargar datos

In [0]:
ruta = 'dbfs:/FileStore/shared_uploads/jgamarramoreno@gmail.com/hack_data.csv'

In [0]:
datos = spark.read.csv(ruta,header=True,inferSchema=True)

In [0]:
datos.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [0]:
datos.head()

Out[6]: Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [0]:
datos.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [0]:
datos.columns

Out[9]: ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

### Formatear datos

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
columnas_caract = ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']

In [0]:
ensamblador = VectorAssembler(inputCols=columnas_caract,outputCol='caracteristicas')

In [0]:
datos_finales = ensamblador.transform(datos)

In [0]:
datos_finales.head()

Out[17]: Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37, caracteristicas=DenseVector([8.0, 391.09, 1.0, 2.96, 7.0, 72.37]))

### Estandarización

In [0]:
from pyspark.ml.feature import StandardScaler

In [0]:
escala = StandardScaler(inputCol='caracteristicas',
                        outputCol='caracteristicasEscala',
                        withStd=True,
                        withMean=False)

In [0]:
modeloEscala = escala.fit(datos_finales)

In [0]:
datos_finales_escala = modeloEscala.transform(datos_finales)

In [0]:
datos_finales_escala.head()

Out[25]: Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37, caracteristicas=DenseVector([8.0, 391.09, 1.0, 2.96, 7.0, 72.37]), caracteristicasEscala=DenseVector([0.5679, 1.3658, 1.9976, 1.2859, 2.2849, 5.3963]))

### Modelos KMeans

In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
kmeans2 = KMeans(featuresCol='caracteristicasEscala',k=2)
kmeans3 = KMeans(featuresCol='caracteristicasEscala',k=3)

In [0]:
modelo_k2 = kmeans2.fit(datos_finales_escala)
modelo_k3 = kmeans3.fit(datos_finales_escala)

In [0]:
# modelo_k2.transform(datos_finales_escala).show()

### Evaluacion de los modelos

In [0]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [0]:
evaluador = ClusteringEvaluator(predictionCol='prediction', featuresCol='caracteristicasEscala',
                                metricName='silhouette', distanceMeasure='squaredEuclidean')

In [0]:
salida_k2 = modelo_k2.transform(datos_finales_escala)
salida_k3 = modelo_k3.transform(datos_finales_escala)

In [0]:
puntaje_k2 = evaluador.evaluate(salida_k2)
puntaje_k3 = evaluador.evaluate(salida_k3)

In [0]:
print("Puntaje con K=2:",puntaje_k2)
print("Puntaje con K=3:",puntaje_k3)

Puntaje con K=2: 0.8176460094012482
Puntaje con K=3: 0.755239186655932


In [0]:
modelo_k2.transform(datos_finales_escala).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



In [0]:
modelo_k3.transform(datos_finales_escala).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   83|
|         2|   84|
|         0|  167|
+----------+-----+



### Conclusión
Hay 2 hackers, los grupos creados tienen el mismo tamaño con k=2

###Tarea: 
Adapte el código que se muestra a continuación para probar el modelo con Ks desde 1 hasta 12, evaluelos y grafíquelo

```
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score=[]
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')
for i in range(2,10):
    
    KMeans_algo=KMeans(featuresCol='standardized', k=i)
    
    KMeans_fit=KMeans_algo.fit(data_scale_output)
    
    output=KMeans_fit.transform(data_scale_output)
    
    
    
    score=evaluator.evaluate(output)
    
    silhouette_score.append(score)
    
    print("Silhouette Score:",score)

#Visualizing the silhouette scores in a plot
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1, figsize =(8,6))
ax.plot(range(2,10),silhouette_score)
ax.set_xlabel(‘k’)
ax.set_ylabel(‘cost’)
```